In [1]:
import pandas as pd
from time import *
from pprint import pprint
import os
current_dir = os.getcwd()

In [67]:
mud_source_path = '/tmp-data/yanhaoyue/workspace/RL/data/mud_new'
feature_source_path = '/tmp-data/yanhaoyue/workspace/RL/data/feature_new'
order_source_path = '/tmp-data/yanhaoyue/workspace/RL/data/order_new'
buffer_store_path = '/tmp-data/yanhaoyue/workspace/RL/data/support_feature_buffer.csv'

## mud

In [3]:
mud = pd.read_csv(mud_source_path, sep='\t', dtype='object')
mud.columns = [c.split(".")[-1] for c in mud.columns]
mud = mud[mud['order_id'].apply(lambda x:len(str(x)))==19]
mud = mud.fillna(-1)

In [4]:
mud.dtypes

order_id      object
maxaction     object
max_qvalue    object
action        object
qvalues       object
waittime      object
ymd           object
rn            object
dtype: object

In [5]:
mud.shape

(1162723, 8)

In [6]:
mud.head(100)

,order_id,maxaction,max_qvalue,action,qvalues,waittime,ymd,rn
0,1441719259641590704,2.210000,0.373863,2.210000,"0.321462,0.345300,0.334358,0.319799,0.364069,0...",205,20231101,1
1,1441719259641590704,1.583000,0.353496,1.583000,"0.322574,0.323923,0.305968,0.324037,0.353496,0...",207,20231101,2
2,1441719259641590704,1.583000,0.375224,1.583000,"0.324030,0.296670,0.289612,0.326526,0.375224,0...",209,20231101,3
3,1441719259641590704,2.210000,0.383570,2.210000,"0.287638,0.301849,0.360628,0.338752,0.294719,0...",211,20231101,4
4,1441719259641590704,1.583000,0.462522,1.583000,"0.274647,0.279902,0.195810,0.278291,0.462522,0...",213,20231101,5
...,...,...,...,...,...,...,...,...
95,1441719259641696626,2.210000,0.468801,2.210000,"0.468064,0.459225,0.210680,0.369555,0.428427,0...",357,20231101,23
96,1441719259641696626,2.210000,0.521722,2.210000,"0.437747,0.404581,0.419833,0.438475,0.504319,0...",359,20231101,24
97,1441719259641696626,2.210000,0.521722,2.210000,"0.437747,0.404581,0.419833,0.438475,0.504319,0...",361,20231101,25
98,1441719259641696626,2.210000,0.521722,2.210000,"0.437747,0.404581,0.419833,0.438475,0.504319,0...",363,20231101,26


In [7]:
mud['order_id'] = mud['order_id'].astype('int')
mud[mud.columns.difference(['order_id', 'qvalues'])] = mud[mud.columns.difference(['order_id', 'qvalues'])].astype(float)

## feature

In [8]:
feature = pd.read_csv(feature_source_path, sep='\t',dtype='object')
feature.columns = [c.split(".")[-1] for c in feature.columns]
feature = feature[feature['f_real_order_id'].apply(lambda x:len(str(x)))==19]
feature = feature[feature['f_trace_id'].apply(lambda x:len(str(x)))==5]
feature = feature.fillna(-1)

In [9]:
feature.shape

(1156923, 434)

In [10]:
feature.drop(columns=['time_stamp', 'createtimestamp', 'f_trace_id', 'b_trace_id', 'm_trace_id'], inplace=True)

In [11]:
id_columns = ['f_real_order_id', 'f_order_id', 'b_real_order_id', 'driver_id', 'm_real_order_id']

feature[id_columns] = feature[id_columns].astype(int)
feature[feature.columns.difference(id_columns)] = feature[feature.columns.difference(id_columns)].astype(float)

In [10]:
# 简易版需要的特征
columns = [
    "f_real_order_id",
    "f_order_id",
    'b_real_order_id',
    'm_real_order_id',
    "driver_id",
    "nearby_total_driver_num", 
    "nearby_total_order_num", 
    "nearby_idle_driver_num", 
    "nearby_grid_order_num", 
    "in_total_driver_num", 
    "in_total_order_num",
    "in_idle_driver_num",
    "in_grid_order_num",
    "pas_dynamic_times",
    "drv_dynamic_times",
    "waittime",
]
feature = feature[columns]

In [12]:
# 现在支持的特征
support_feature = pd.read_csv('/tmp-data/yanhaoyue/workspace/RL/data/support_feature.csv', \
                              sep=',', dtype='object')
feature_columns = [    
    "f_real_order_id",
    "f_order_id",
    'b_real_order_id',
    'm_real_order_id',
    "driver_id",
    "waittime"
]
for feature_name in support_feature[support_feature['status'] == 'done']['特征名称']:
    feature_columns.append(feature_name)

In [13]:
feature = feature[feature_columns]

In [ ]:
# 完整版需要的特征

In [14]:
feature['f_order_id'].value_counts()

f_order_id
87996128863879    239
87996125500460    232
87996128495030    230
87996129153687    217
87996110011942    210
                 ... 
87996139907160      1
87996148410813      1
87996137481504      1
87996114396432      1
87996111247457      1
Name: count, Length: 212314, dtype: int64

In [15]:
feature.dtypes

f_real_order_id                                                          int64
f_order_id                                                               int64
b_real_order_id                                                          int64
m_real_order_id                                                          int64
driver_id                                                                int64
                                                                        ...   
dest_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg    float64
dest_driver_strive_zhipai_order_minute_cnt_5                           float64
start_driver_strive_zhipai_order_minute_cnt                            float64
dest_driver_strive_zhipai_order_minute_cnt                             float64
start_biz_passengerid_order_finish_minute_pidlistcnt                   float64
Length: 114, dtype: object

## order

In [16]:
order = pd.read_csv(order_source_path, sep='\t', dtype='object')
order.columns = [c.split(".")[-1] for c in order.columns]
order = order[order['order_id'].apply(lambda x:len(str(x)))==14]
order.drop(columns=['call_time'], inplace=True)
order = order.fillna(-1)

In [17]:
order.shape

(436667, 13)

In [18]:
order.dtypes

order_id                                               object
driver_id                                              object
calls                                                  object
rides                                                  object
acceptances                                            object
calls_cancelled_before_accepted                        object
calls_cancelled_by_passenger_after_accepted            object
calls_cancelled_by_driver_after_accepted               object
calls_cancelled_by_drivers_after_accepted_addresend    object
serial_calls                                           object
serial_rides                                           object
serial_acceptances                                     object
serial_pcaa_cnt                                        object
dtype: object

In [19]:
order.head(100)

,order_id,driver_id,calls,rides,acceptances,calls_cancelled_before_accepted,calls_cancelled_by_passenger_after_accepted,calls_cancelled_by_driver_after_accepted,calls_cancelled_by_drivers_after_accepted_addresend,serial_calls,serial_rides,serial_acceptances,serial_pcaa_cnt
0,87996164412943,650911395469613,1,1,1,0,0,0,0,0,0,0,0
1,87996170208349,633318717922924,1,1,1,0,0,0,0,0,0,0,0
2,87996161725802,0,1,0,0,1,0,0,0,0,0,0,0
3,87996159704974,650911459813961,0,1,0,0,0,0,0,0,0,0,0
4,87996171818577,633318719314948,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,87996170979597,650911033150566,1,1,1,0,0,0,0,0,0,0,0
96,87996176126712,650910969865467,1,1,1,0,0,0,0,0,0,0,0
97,87996170951169,650910929463003,0,1,0,0,0,0,0,0,1,0,0
98,87996169981318,650910963774048,1,0,1,0,0,0,1,1,0,1,0


order_driver_id 为0说明没有司机接单，或者说是乘客取消了

In [22]:
# order['driver_id'].replace('0', '-1', inplace=True)

In [20]:
order = order.astype('int64')

## merge

feature order merge

In [21]:
feature = feature.rename(columns={'driver_id': 'feature_driver_id'})

In [22]:
feature.columns

Index(['f_real_order_id', 'f_order_id', 'b_real_order_id', 'm_real_order_id',
       'feature_driver_id', 'waittime', 'nearby_total_driver_num',
       'nearby_total_order_num', 'nearby_idle_driver_num',
       'nearby_grid_order_num',
       ...
       'driver_driver_strive_zhipai_order_minute_cnt',
       'start_driver_strive_zhipai_order_minute_cnt_10',
       'asdest_driver_strive_zhipai_order_minute_cnt_10',
       'start_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg',
       'start_driver_strive_zhipai_order_minute_cnt_5',
       'dest_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg',
       'dest_driver_strive_zhipai_order_minute_cnt_5',
       'start_driver_strive_zhipai_order_minute_cnt',
       'dest_driver_strive_zhipai_order_minute_cnt',
       'start_biz_passengerid_order_finish_minute_pidlistcnt'],
      dtype='object', length=114)

In [23]:
order.columns

Index(['order_id', 'driver_id', 'calls', 'rides', 'acceptances',
       'calls_cancelled_before_accepted',
       'calls_cancelled_by_passenger_after_accepted',
       'calls_cancelled_by_driver_after_accepted',
       'calls_cancelled_by_drivers_after_accepted_addresend', 'serial_calls',
       'serial_rides', 'serial_acceptances', 'serial_pcaa_cnt'],
      dtype='object')

联表feature, order; 联表后在处理calls_cancelled_before_accepted的done标志设置的问题

In [24]:
feature_order_merge = feature.merge(order[['order_id', 'driver_id', 'rides', \
                                           'calls_cancelled_before_accepted', \
                                          'calls_cancelled_by_passenger_after_accepted', \
                                          'calls_cancelled_by_drivers_after_accepted_addresend']], \
                                    left_on=['f_order_id', 'feature_driver_id'], \
                                    right_on=['order_id', 'driver_id'], \
                                    how='left')

In [25]:
feature_order_merge.dtypes

f_real_order_id                                          int64
f_order_id                                               int64
b_real_order_id                                          int64
m_real_order_id                                          int64
feature_driver_id                                        int64
                                                        ...   
driver_id                                              float64
rides                                                  float64
calls_cancelled_before_accepted                        float64
calls_cancelled_by_passenger_after_accepted            float64
calls_cancelled_by_drivers_after_accepted_addresend    float64
Length: 120, dtype: object

In [131]:
feature_order_merge.describe()

,f_real_order_id,f_order_id,b_real_order_id,m_real_order_id,feature_driver_id,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,in_total_driver_num,...,in_grid_order_num,pas_dynamic_times,drv_dynamic_times,waittime,order_id,driver_id,rides,calls_cancelled_before_accepted,calls_cancelled_by_passenger_after_accepted,calls_cancelled_by_drivers_after_accepted_addresend
count,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,...,1.156923e+06,1.156923e+06,1.156923e+06,1.156923e+06,6.590100e+04,6.590100e+04,65901.000000,65901.0,65901.000000,65901.000000
mean,1.441721e+18,8.799613e+13,4.334859e+17,1.441721e+18,1.952292e+14,2.443107e+02,1.463949e+01,9.082357e+01,4.019072e+00,4.000603e+02,...,3.927361e+01,1.183250e-01,1.154399e-01,8.294398e+01,8.799613e+13,6.487275e+14,0.717318,0.0,0.051410,0.231119
std,7.332280e+11,1.115299e+07,6.611021e+17,7.332280e+11,2.977537e+14,1.084585e+02,1.188869e+01,4.904792e+01,3.573723e+00,1.308789e+02,...,2.406798e+01,2.263866e-01,2.227192e-01,9.462922e+01,5.438293e+06,5.800697e+12,0.450306,0.0,0.220835,0.421552
min,1.441719e+18,8.799611e+13,-1.000000e+00,1.441719e+18,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.799612e+13,6.333187e+14,0.000000,0.0,0.000000,0.000000
25%,1.441721e+18,8.799612e+13,-1.000000e+00,1.441721e+18,-1.000000e+00,1.600000e+02,6.000000e+00,5.500000e+01,1.000000e+00,3.420000e+02,...,2.000000e+01,0.000000e+00,0.000000e+00,1.600000e+01,8.799613e+13,6.509109e+14,0.000000,0.0,0.000000,0.000000
50%,1.441721e+18,8.799613e+13,-1.000000e+00,1.441721e+18,-1.000000e+00,2.570000e+02,1.100000e+01,8.400000e+01,3.000000e+00,4.560000e+02,...,3.500000e+01,0.000000e+00,0.000000e+00,5.500000e+01,8.799613e+13,6.509111e+14,1.000000,0.0,0.000000,0.000000
75%,1.441721e+18,8.799614e+13,1.441721e+18,1.441721e+18,6.509109e+14,3.240000e+02,2.100000e+01,1.170000e+02,5.000000e+00,5.000000e+02,...,5.500000e+01,1.100000e-01,1.000000e-01,1.150000e+02,8.799614e+13,6.509113e+14,1.000000,0.0,0.000000,0.000000
max,1.441721e+18,8.799615e+13,1.441721e+18,1.441721e+18,6.509115e+14,5.000000e+02,7.500000e+01,3.710000e+02,2.600000e+01,5.000000e+02,...,1.410000e+02,3.000000e+00,3.000000e+00,7.160000e+02,8.799614e+13,6.509115e+14,1.000000,0.0,1.000000,1.000000


In [26]:
feature_order_merge = feature_order_merge.assign(reward=0.0, done=0.0) # 增加了两列
feature_order_merge.fillna(-1)

,f_real_order_id,f_order_id,b_real_order_id,m_real_order_id,feature_driver_id,waittime,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,...,dest_driver_strive_zhipai_order_minute_cnt,start_biz_passengerid_order_finish_minute_pidlistcnt,order_id,driver_id,rides,calls_cancelled_before_accepted,calls_cancelled_by_passenger_after_accepted,calls_cancelled_by_drivers_after_accepted_addresend,reward,done
0,1441721012003706411,87996123747883,-1,1441721012003706411,-1,283.0,483.0,14.0,128.0,6.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
1,1441721012003314474,87996123355946,1441721012003314474,1441721012003314474,650911133876009,57.0,321.0,27.0,97.0,9.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
2,1441721012011434033,87996131475505,-1,1441721012011434033,-1,130.0,313.0,7.0,125.0,3.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
3,1441721012017148583,87996137190055,1441721012017148583,1441721012017148583,650911370812297,95.0,254.0,9.0,97.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
4,1441721012008893342,87996128934814,-1,1441721012008893342,-1,123.0,175.0,26.0,75.0,9.0,...,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156918,1441719259663434172,87996130132412,-1,1441719259663434172,-1,82.0,175.0,16.0,41.0,2.0,...,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
1156919,1441721012013448419,87996133489891,-1,1441721012013448419,-1,1.0,428.0,4.0,148.0,1.0,...,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
1156920,1441721011993822817,87996113864289,-1,1441721011993822817,-1,19.0,282.0,40.0,69.0,3.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0
1156921,1441721011998630279,87996118671751,-1,1441721011998630279,-1,112.0,173.0,12.0,77.0,5.0,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0


In [27]:
# 没有播单的，reward为0
feature_order_merge.loc[feature_order_merge['b_real_order_id'] == -1, 'reward'] = 0
feature_order_merge.loc[feature_order_merge['b_real_order_id'] == -1, 'done'] = 0

# 播单了但是没有司机接的，reward为-0.05
feature_order_merge.loc[(feature_order_merge['b_real_order_id'] != -1) & \
                        (feature_order_merge['calls_cancelled_before_accepted'] != 1) & \
                        (feature_order_merge['driver_id'] != feature_order_merge['feature_driver_id']), 'reward'] = -0.05
feature_order_merge.loc[(feature_order_merge['b_real_order_id'] != -1) & \
                        (feature_order_merge['calls_cancelled_before_accepted'] != 1) & \
                        (feature_order_merge['driver_id'] != feature_order_merge['feature_driver_id']), 'done'] = 0

# 乘客应答后取消，reward为0
feature_order_merge.loc[(feature_order_merge['calls_cancelled_by_passenger_after_accepted'] == 1) & \
                        (feature_order_merge['driver_id'] == feature_order_merge['feature_driver_id']) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'reward'] = 0
feature_order_merge.loc[(feature_order_merge['calls_cancelled_by_passenger_after_accepted'] == 1) & \
                        (feature_order_merge['driver_id'] == feature_order_merge['feature_driver_id']) & 
                        (feature_order_merge['feature_driver_id'] != -1), 'done'] = 1

# 司机应答后取消，reward为0
feature_order_merge.loc[(feature_order_merge['calls_cancelled_by_drivers_after_accepted_addresend'] == 1) & \
                        (feature_order_merge['driver_id'] == feature_order_merge['feature_driver_id']) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'reward'] = 0
feature_order_merge.loc[(feature_order_merge['calls_cancelled_by_drivers_after_accepted_addresend'] == 1) & \
                        (feature_order_merge['driver_id'] == feature_order_merge['feature_driver_id']) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'done'] = 1

# 完单，reward为1
feature_order_merge.loc[(feature_order_merge['rides'] == 1) & \
                        (feature_order_merge['driver_id'] == feature_order_merge['feature_driver_id']) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'reward'] = 1  
feature_order_merge.loc[(feature_order_merge['rides'] == 1) & \
                        (feature_order_merge['driver_id'] == feature_order_merge['feature_driver_id']) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'done'] = 1

# 乘客应答前取消
feature_order_merge.loc[(feature_order_merge['calls_cancelled_before_accepted'] == 1) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'reward'] = 0
feature_order_merge.loc[(feature_order_merge['calls_cancelled_before_accepted'] == 1) & \
                        (feature_order_merge['feature_driver_id'] != -1), 'done'] = 1

乘客应答前取消需要重新计算， 找出feature_order_merge表中， 字段 f_order_id 相同的那些组里面， <br />
字段waittime最大的是在feature_order_merge的第几行<br/>
现在可以按照f_order_id concat 拼接长度（高位补0）的waittime，然后sort

In [28]:
feature_order_merge = feature_order_merge.assign(group_sort_key=0)

In [29]:
feature_order_merge['waittime'] = feature_order_merge['waittime'].astype('int')

In [30]:
feature_order_merge.group_sort_key = \
        (feature_order_merge['f_order_id'].astype(str) + \
         feature_order_merge['waittime'].astype(str).str.zfill(4)).astype('int')

In [31]:
feature_order_merge.sort_values('group_sort_key')[['f_real_order_id', 'waittime']] # 实际上没有被赋值，只是验证了可行性

,f_real_order_id,waittime
46363,1441721011988188035,296
731285,1441721011988188035,298
750208,1441721011988188035,300
775589,1441721011988190924,385
467576,1441721011988190924,424
...,...,...
1020899,1441721012029433285,2
640794,1441721012029450765,1
961491,1441719259682814125,1
969122,1441719259682814125,15


这里有一个假设的前提，feature_order_group后的每个order如果最后的状态是CBA，那么一定会在order表中查询出calls_cancelled_before_accepted为1

In [32]:
order['order_id'][order['calls_cancelled_before_accepted'] == 1]

2         87996161725802
15        87996133356473
19        87996130351381
32        87996131369138
39        87996109367642
               ...      
436597    87996148630190
436599    87996163564336
436609    87996117824433
436626    87996152241925
436638    87996118780089
Name: order_id, Length: 43155, dtype: int64

In [33]:
process_indicator = 0
for CBA_id in order['order_id'][order['calls_cancelled_before_accepted'] == 1]: # CBA的order_id
    process_indicator += 1
    if process_indicator % 100 == 0:
        print('now process: ', process_indicator / 43155)
    if not feature_order_merge[feature_order_merge['f_order_id'] == CBA_id]['waittime'].empty:
        idx = feature_order_merge[feature_order_merge['f_order_id'] == CBA_id]['waittime'].last_valid_index()
        feature_order_merge.loc[idx, 'done'] = 1.0
print("process done")

now process:  0.0023172285946008574
now process:  0.004634457189201715
now process:  0.006951685783802572
now process:  0.00926891437840343
now process:  0.011586142973004287
now process:  0.013903371567605145
now process:  0.016220600162206
now process:  0.01853782875680686
now process:  0.020855057351407715
now process:  0.023172285946008574
now process:  0.02548951454060943
now process:  0.02780674313521029
now process:  0.030123971729811145
now process:  0.032441200324412
now process:  0.03475842891901286
now process:  0.03707565751361372
now process:  0.03939288610821458
now process:  0.04171011470281543
now process:  0.04402734329741629
now process:  0.04634457189201715
now process:  0.04866180048661801
now process:  0.05097902908121886
now process:  0.05329625767581972
now process:  0.05561348627042058
now process:  0.05793071486502144
now process:  0.06024794345962229
now process:  0.06256517205422316
now process:  0.064882400648824
now process:  0.06719962924342486
now process

now process:  0.5723554628664118
now process:  0.5746726914610126
now process:  0.5769899200556134
now process:  0.5793071486502144
now process:  0.5816243772448152
now process:  0.583941605839416
now process:  0.5862588344340169
now process:  0.5885760630286178
now process:  0.5908932916232186
now process:  0.5932105202178195
now process:  0.5955277488124203
now process:  0.5978449774070212
now process:  0.6001622060016221
now process:  0.6024794345962229
now process:  0.6047966631908238
now process:  0.6071138917854246
now process:  0.6094311203800254
now process:  0.6117483489746264
now process:  0.6140655775692272
now process:  0.616382806163828
now process:  0.6187000347584289
now process:  0.6210172633530298
now process:  0.6233344919476307
now process:  0.6256517205422315
now process:  0.6279689491368323
now process:  0.6302861777314333
now process:  0.6326034063260341
now process:  0.6349206349206349
now process:  0.6372378635152358
now process:  0.6395550921098366
now process:

In [34]:
feature_order_merge.dtypes

f_real_order_id                                          int64
f_order_id                                               int64
b_real_order_id                                          int64
m_real_order_id                                          int64
feature_driver_id                                        int64
                                                        ...   
calls_cancelled_by_passenger_after_accepted            float64
calls_cancelled_by_drivers_after_accepted_addresend    float64
reward                                                 float64
done                                                   float64
group_sort_key                                           int64
Length: 123, dtype: object

到这里feature_order_merge成功了

In [35]:
feature_order_merge.drop(columns=['order_id'], inplace=True) # 避免与mud的order_id重复

In [36]:
feature_order_mud_merge = feature_order_merge.merge(mud, \
                                                    left_on=['f_real_order_id', 'waittime'],\
                                                    right_on=['order_id', 'waittime'],
                                                    how='inner'
                                                   )

In [37]:
feature_order_mud_merge.fillna(-1)

,f_real_order_id,f_order_id,b_real_order_id,m_real_order_id,feature_driver_id,waittime,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,...,reward,done,group_sort_key,order_id,maxaction,max_qvalue,action,qvalues,ymd,rn
0,1441721012003314474,87996123355946,1441721012003314474,1441721012003314474,650911133876009,57,321.0,27.0,97.0,9.0,...,-0.05,0.0,879961233559460057,1441721012003314474,2.210,0.000000,0.412,"0.000000,0.000000,0.000000,0.000000,0.000000,0...",20231101.0,6.0
1,1441721012004246964,87996124288436,-1,1441721012004246964,-1,113,246.0,8.0,107.0,1.0,...,0.00,0.0,879961242884360113,1441721012004246964,2.210,0.537753,2.210,"0.469438,0.335254,0.265444,0.455315,0.532462,0...",20231101.0,31.0
2,1441721012007513826,87996127555298,-1,1441721012007513826,-1,395,387.0,8.0,112.0,1.0,...,0.00,0.0,879961275552980395,1441721012007513826,2.210,0.368119,2.210,"0.323665,0.277283,0.277456,0.282394,0.273140,0...",20231101.0,127.0
3,1441721012004911234,87996124952706,1441721012004911234,1441721012004911234,650911060993602,2,379.0,15.0,152.0,3.0,...,1.00,1.0,879961249527060002,1441721012004911234,0.003,0.794978,0.003,"0.794978,0.726824,0.493769,0.542491,0.629853,0...",20231101.0,1.0
4,1441721012026825080,87996146866552,-1,1441721012026825080,-1,44,291.0,9.0,153.0,3.0,...,0.00,0.0,879961468665520044,1441721012026825080,0.992,0.543145,0.992,"0.373641,0.503611,0.363637,0.543145,0.529774,0...",20231102.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493228,1441721012022816601,87996142858073,-1,1441721012022816601,-1,14,104.0,13.0,70.0,6.0,...,0.00,0.0,879961428580730014,1441721012022816601,0.161,0.521662,0.161,"0.361916,0.521662,0.516983,0.403592,0.500221,0...",20231102.0,2.0
493229,1441721011989581276,87996109622748,-1,1441721011989581276,-1,331,21.0,2.0,17.0,1.0,...,0.00,0.0,879961096227480331,1441721011989581276,2.210,0.533785,2.210,"0.409246,0.443795,0.529958,0.516183,0.489890,0...",20231101.0,7.0
493230,1441721012018406882,87996138448354,-1,1441721012018406882,-1,24,117.0,2.0,55.0,1.0,...,0.00,0.0,879961384483540024,1441721012018406882,0.003,0.709893,0.003,"0.709893,0.469746,0.680644,0.583101,0.593108,0...",20231102.0,2.0
493231,1441721011993822817,87996113864289,-1,1441721011993822817,-1,19,282.0,40.0,69.0,3.0,...,0.00,0.0,879961138642890019,1441721011993822817,0.161,0.722209,0.161,"0.557281,0.722209,0.586019,0.582038,0.466277,0...",20231101.0,3.0


In [38]:
# 先不执行，等待下面测试，流程走完毕
mud['waittime'] = mud['waittime'] - 1 # 同一个order进入mud的时间比feature的时间晚，因此mud的时间-1，再进行匹配

In [39]:
# 先不执行，等待下面测试，流程走完毕
feature_order_mud_shift_waittime_merge = feature_order_merge.merge(mud, \
                                                                left_on=['f_real_order_id', 'waittime'],\
                                                                right_on=['order_id', 'waittime'],
                                                                how='inner'
                                                               )

In [40]:
feature_order_mud_shift_waittime_merge.fillna(-1)

,f_real_order_id,f_order_id,b_real_order_id,m_real_order_id,feature_driver_id,waittime,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,...,reward,done,group_sort_key,order_id,maxaction,max_qvalue,action,qvalues,ymd,rn
0,1441721012003706411,87996123747883,-1,1441721012003706411,-1,283,483.0,14.0,128.0,6.0,...,0.00,0.0,879961237478830283,1441721012003706411,2.210,0.414038,2.210,"0.378625,0.322014,0.374120,0.378242,0.335807,0...",20231101.0,51.0
1,1441721012011434033,87996131475505,-1,1441721012011434033,-1,130,313.0,7.0,125.0,3.0,...,0.00,0.0,879961314755050130,1441721012011434033,2.210,0.448647,2.210,"0.391131,0.186263,0.369905,0.406312,0.427046,0...",20231102.0,38.0
2,1441721012017148583,87996137190055,1441721012017148583,1441721012017148583,650911370812297,95,254.0,9.0,97.0,1.0,...,-0.05,0.0,879961371900550095,1441721012017148583,2.210,0.621556,2.210,"0.344750,0.586527,0.341770,0.368969,0.370745,0...",20231102.0,6.0
3,1441721012008893342,87996128934814,-1,1441721012008893342,-1,123,175.0,26.0,75.0,9.0,...,0.00,0.0,879961289348140123,1441721012008893342,0.992,0.373788,0.992,"0.323884,0.313652,0.311785,0.373788,0.353246,0...",20231101.0,27.0
4,1441721011994424088,87996114465560,-1,1441721011994424088,-1,77,330.0,9.0,98.0,2.0,...,0.00,0.0,879961144655600077,1441721011994424088,0.003,0.621080,0.003,"0.621080,0.418514,0.508212,0.437290,0.478934,0...",20231101.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403548,1441721012010672432,87996130713904,1441721012010672432,1441721012010672432,650910915870812,69,280.0,9.0,105.0,4.0,...,1.00,1.0,879961307139040069,1441721012010672432,0.003,0.564667,1.583,"0.564667,0.452587,0.340017,0.441597,0.364171,0...",20231102.0,6.0
403549,1441721011993588958,87996113630430,-1,1441721011993588958,-1,128,371.0,29.0,120.0,5.0,...,0.00,0.0,879961136304300128,1441721011993588958,1.583,0.535969,1.583,"0.366958,0.505040,0.365656,0.414754,0.535969,0...",20231101.0,15.0
403550,1441721011998033483,87996118074955,-1,1441721011998033483,-1,5,240.0,14.0,125.0,5.0,...,0.00,0.0,879961180749550005,1441721011998033483,1.583,0.742774,1.583,"0.503333,0.614491,0.506226,0.472212,0.742774,0...",20231101.0,1.0
403551,1441719259663434172,87996130132412,-1,1441719259663434172,-1,82,175.0,16.0,41.0,2.0,...,0.00,0.0,879961301324120082,1441719259663434172,2.210,0.533411,2.210,"0.360327,0.488976,0.355910,0.349159,0.291377,0...",20231102.0,11.0


In [41]:
feature_order_mud = pd.concat([feature_order_mud_merge, feature_order_mud_shift_waittime_merge])

In [42]:
feature_order_mud.group_sort_key = \
        (feature_order_mud['f_order_id'].astype(str) + \
         feature_order_mud['waittime'].astype(str).str.zfill(4)).astype('int')

In [43]:
feature_order_mud = feature_order_mud.sort_values('group_sort_key')

In [44]:
feature_order_mud.head()

,f_real_order_id,f_order_id,b_real_order_id,m_real_order_id,feature_driver_id,waittime,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,...,reward,done,group_sort_key,order_id,maxaction,max_qvalue,action,qvalues,ymd,rn
16347,1441721011988188035,87996108229507,-1,1441721011988188035,-1,296,61.0,4.0,39.0,2.0,...,0.0,0.0,879961082295070296,1441721011988188035,0.992,0.438742,0.992,"0.353952,0.369475,0.396115,0.438742,0.404616,0...",20231101.0,1.0
256237,1441721011988188035,87996108229507,-1,1441721011988188035,-1,298,61.0,4.0,39.0,2.0,...,0.0,0.0,879961082295070298,1441721011988188035,0.992,0.438742,0.992,"0.353952,0.369475,0.396115,0.438742,0.404616,0...",20231101.0,2.0
262739,1441721011988188035,87996108229507,-1,1441721011988188035,-1,300,55.0,3.0,37.0,2.0,...,0.0,0.0,879961082295070300,1441721011988188035,1.583,0.626766,1.583,"0.452776,0.476869,0.462344,0.509384,0.626766,0...",20231101.0,3.0
332232,1441721011988190924,87996108232396,-1,1441721011988190924,-1,385,197.0,6.0,49.0,1.0,...,0.0,0.0,879961082323960385,1441721011988190924,0.992,0.510896,0.992,"0.502698,0.425686,0.451434,0.510896,0.506011,0...",20231101.0,1.0
200939,1441721011988190924,87996108232396,-1,1441721011988190924,-1,424,166.0,11.0,42.0,3.0,...,0.0,0.0,879961082323960424,1441721011988190924,0.161,0.470580,0.161,"0.353065,0.470580,0.393247,0.470533,0.425024,0...",20231101.0,2.0


## buffer

(s, a, r, s', done) 简易版特征

In [72]:
columns = [
    "f_real_order_id",
    "f_order_id",
    "nearby_total_driver_num", 
    "nearby_total_order_num", 
    "nearby_idle_driver_num", 
    "nearby_grid_order_num", 
    "in_total_driver_num", 
    "in_total_order_num",
    "in_idle_driver_num",
    "in_grid_order_num",
    "pas_dynamic_times",
    "drv_dynamic_times",
    "waittime",
    "action",
    "reward",
    "done"
]


s = [
    "nearby_total_driver_num", 
    "nearby_total_order_num", 
    "nearby_idle_driver_num", 
    "nearby_grid_order_num", 
    "in_total_driver_num", 
    "in_total_order_num",
    "in_idle_driver_num",
    "in_grid_order_num",
    "pas_dynamic_times",
    "drv_dynamic_times",
    "waittime"
]

a = "action"
r = "reward"
d = "done"

(s, a, r, s', done) 全版特征

In [49]:
# 需要剔除的特征有
# m_real_order_id
# f_order_id
# driver_id, 
# rides,
# calls_cancelled_before_accepted
# calls_cancelled_by_drivers_after_accepted_addresend
# calls_cancelled_by_passenger_after_accepted
# group_sort_key
# order_id
# maxaction
# max_qvalue
# dispatch_fee # 全为0
# qvalues
# ymd
# rn
# b_real_order_id
# feature_driver_id
import copy
columns = [column for column in feature_order_mud.columns]
for remove_column in [
    'm_real_order_id',
    'f_order_id',
    'driver_id',
    'rides',
    'calls_cancelled_before_accepted',
    'calls_cancelled_by_drivers_after_accepted_addresend',
    'calls_cancelled_by_passenger_after_accepted',
    'group_sort_key',
    'order_id',
    'maxaction',
    'max_qvalue',
    'qvalues',
    'ymd',
    'rn',
    'b_real_order_id',
    'feature_driver_id',
]:
    columns.remove(remove_column)
    
s = copy.deepcopy(columns)
for remove_column in [
    'f_real_order_id',
    'action',
    'reward',
    'done'
]:
    s.remove(remove_column)

a = "action"
r = "reward"
d = "done"

In [50]:
states = feature_order_mud[columns]

In [51]:
states.reset_index(inplace=True)

In [52]:
states.drop(columns=['index'], inplace=True)

/tmp/ipykernel_3527663/2504869330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  states.drop(columns=['index'], inplace=True)


In [53]:
states.head()

,f_real_order_id,waittime,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,driver_coordstream_driver_nonempty_time_minute_avg_10,driver_coordstream_driver_empty_time_minute_avg_5,driver_coordstream_driver_online_service_time_minute_avg_10,driver_coordstream_driver_empty_time_minute_avg_10,...,start_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg,start_driver_strive_zhipai_order_minute_cnt_5,dest_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg,dest_driver_strive_zhipai_order_minute_cnt_5,start_driver_strive_zhipai_order_minute_cnt,dest_driver_strive_zhipai_order_minute_cnt,start_biz_passengerid_order_finish_minute_pidlistcnt,reward,done,action
0,1441721011988188035,296,61.0,4.0,39.0,2.0,2.455,2.49,3.080,3.125,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.992
1,1441721011988188035,298,61.0,4.0,39.0,2.0,2.455,2.49,3.080,3.125,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.992
2,1441721011988188035,300,55.0,3.0,37.0,2.0,2.455,2.49,3.080,3.125,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.583
3,1441721011988190924,385,197.0,6.0,49.0,1.0,2.000,2.00,2.500,2.000,...,-1.0,2.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,0.0,0.992
4,1441721011988190924,424,166.0,11.0,42.0,3.0,2.960,1.50,3.155,2.750,...,-1.0,2.0,13.0,3.0,-1.0,3.0,-1.0,0.0,0.0,0.161


In [54]:
for name, max_score in zip(states.columns, states[s].max()):
    print(name, max_score)

f_real_order_id 716.0
waittime 500.0
nearby_total_driver_num 75.0
nearby_total_order_num 339.0
nearby_idle_driver_num 26.0
nearby_grid_order_num 10.0
driver_coordstream_driver_nonempty_time_minute_avg_10 10.0
driver_coordstream_driver_empty_time_minute_avg_5 13.0
driver_coordstream_driver_online_service_time_minute_avg_10 12.0
driver_coordstream_driver_empty_time_minute_avg_10 32.0
driver_engine_order_broadcast_count_minute_avg_10 6.0
driver_coordstream_driver_nonempty_time_minute_avg_5 55.0
driver_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg_10 1823.0
driver_passenger_wait_time_minute_avg_10 12.0
dest_coordstream_driver_online_service_time_minute_avg_10 9.0
driver_coordstream_driver_online_service_time_minute_avg_5 11.0
dest_coordstream_driver_nonempty_time_minute_avg_10 46.0
start_engine_order_broadcast_count_minute_avg_10 14.0
dest_coordstream_driver_empty_time_minute_avg_10 1823.0
dest_passenger_wait_time_minute_avg_10 8.0
dest_coordstream_driver_nonempty_time_min

In [55]:
# 最大值归一化
states.loc[:, s] = states[s] / states[s].max()

In [56]:
states.head()

,f_real_order_id,waittime,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,driver_coordstream_driver_nonempty_time_minute_avg_10,driver_coordstream_driver_empty_time_minute_avg_5,driver_coordstream_driver_online_service_time_minute_avg_10,driver_coordstream_driver_empty_time_minute_avg_10,...,start_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg,start_driver_strive_zhipai_order_minute_cnt_5,dest_biz_zhipai_order_broadcast_page_driver_waiting_time_minute_avg,dest_driver_strive_zhipai_order_minute_cnt_5,start_driver_strive_zhipai_order_minute_cnt,dest_driver_strive_zhipai_order_minute_cnt,start_biz_passengerid_order_finish_minute_pidlistcnt,reward,done,action
0,1441721011988188035,0.413408,0.122,0.053333,0.115044,0.076923,0.2455,0.249,0.236923,0.260417,...,-0.012987,0.028571,-0.013889,-0.028571,-0.083333,-0.090909,-0.090909,0.0,0.0,0.992
1,1441721011988188035,0.416201,0.122,0.053333,0.115044,0.076923,0.2455,0.249,0.236923,0.260417,...,-0.012987,0.028571,-0.013889,-0.028571,-0.083333,-0.090909,-0.090909,0.0,0.0,0.992
2,1441721011988188035,0.418994,0.110,0.040000,0.109145,0.076923,0.2455,0.249,0.236923,0.260417,...,-0.012987,0.028571,-0.013889,-0.028571,-0.083333,-0.090909,-0.090909,0.0,0.0,1.583
3,1441721011988190924,0.537709,0.394,0.080000,0.144543,0.038462,0.2000,0.200,0.192308,0.166667,...,-0.012987,0.057143,-0.013889,0.028571,-0.083333,-0.090909,-0.090909,0.0,0.0,0.992
4,1441721011988190924,0.592179,0.332,0.146667,0.123894,0.115385,0.2960,0.150,0.242692,0.229167,...,-0.012987,0.057143,0.180556,0.085714,-0.083333,0.272727,-0.090909,0.0,0.0,0.161


In [57]:
states.shape

(896786, 113)

In [58]:
buffer = pd.DataFrame(columns=['id', 'wait', 's', 'a', 'r', 's_', 'd'], index=states.index)

buffer['id'] = states['f_real_order_id']
buffer['wait'] = states['waittime']
buffer['s'] = states[s].values.tolist()
buffer['a'] = states[a]
buffer['r'] = states[r]
buffer['s_'] = states[s].shift(-1).fillna(states[s].iloc[-1]).values.tolist()
buffer['d'] = states[d]

In [59]:
buffer['s_'].head()

0    [0.41620111731843573, 0.122, 0.053333333333333...
1    [0.41899441340782123, 0.11, 0.04, 0.1091445427...
2    [0.5377094972067039, 0.394, 0.08, 0.1445427728...
3    [0.5921787709497207, 0.332, 0.1466666666666666...
4    [0.6284916201117319, 0.338, 0.1466666666666666...
Name: s_, dtype: object

In [60]:
buffer = buffer.reset_index()

In [61]:
buffer.head()

,index,id,wait,s,a,r,s_,d
0,0,1441721011988188035,0.413408,"[0.4134078212290503, 0.122, 0.0533333333333333...",0.992,0.0,"[0.41620111731843573, 0.122, 0.053333333333333...",0.0
1,1,1441721011988188035,0.416201,"[0.41620111731843573, 0.122, 0.053333333333333...",0.992,0.0,"[0.41899441340782123, 0.11, 0.04, 0.1091445427...",0.0
2,2,1441721011988188035,0.418994,"[0.41899441340782123, 0.11, 0.04, 0.1091445427...",1.583,0.0,"[0.5377094972067039, 0.394, 0.08, 0.1445427728...",0.0
3,3,1441721011988190924,0.537709,"[0.5377094972067039, 0.394, 0.08, 0.1445427728...",0.992,0.0,"[0.5921787709497207, 0.332, 0.1466666666666666...",0.0
4,4,1441721011988190924,0.592179,"[0.5921787709497207, 0.332, 0.1466666666666666...",0.161,0.0,"[0.6284916201117319, 0.338, 0.1466666666666666...",0.0


In [62]:
# 使用条件索引找出当前行 id 不等于下一行 id 的行
mask = buffer['id'] != buffer['id'].shift(-1)
# 每一个id的最后一个，是可以没有下一个状态的
mask[buffer['d'] == 1] = False
# 最后一行单独判断, 如果是done == 1保留，否则删除
if buffer.iloc[-1]['d'] == 1:
    mask[-1] = False
else:
    mask[-1] = True
buffer.drop(buffer[mask].index, inplace=True)

/tmp/ipykernel_3527663/2423761383.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buffer.drop(buffer[mask].index, inplace=True)


In [63]:
buffer.drop(columns=['id', 'wait'], inplace=True)

In [64]:
buffer.loc[buffer['a'] == 0.003, 'a'] = 0
buffer.loc[buffer['a'] == 0.161, 'a'] = 1
buffer.loc[buffer['a'] == 0.412, 'a'] = 2
buffer.loc[buffer['a'] == 0.992, 'a'] = 3
buffer.loc[buffer['a'] == 1.583, 'a'] = 4
buffer.loc[buffer['a'] == 2.210, 'a'] = 5

In [65]:
buffer['a'].value_counts()

a
5.0    234962
4.0    179231
3.0    124849
2.0    111761
0.0     93309
1.0     93183
Name: count, dtype: int64

In [66]:
buffer.head()

,index,s,a,r,s_,d
0,0,"[0.4134078212290503, 0.122, 0.0533333333333333...",3.0,0.0,"[0.41620111731843573, 0.122, 0.053333333333333...",0.0
1,1,"[0.41620111731843573, 0.122, 0.053333333333333...",3.0,0.0,"[0.41899441340782123, 0.11, 0.04, 0.1091445427...",0.0
3,3,"[0.5377094972067039, 0.394, 0.08, 0.1445427728...",3.0,0.0,"[0.5921787709497207, 0.332, 0.1466666666666666...",0.0
4,4,"[0.5921787709497207, 0.332, 0.1466666666666666...",1.0,0.0,"[0.6284916201117319, 0.338, 0.1466666666666666...",0.0
5,5,"[0.6284916201117319, 0.338, 0.1466666666666666...",5.0,0.0,"[0.6312849162011173, 0.338, 0.1466666666666666...",0.0


In [68]:
buffer.to_csv(buffer_store_path, sep='\t', index=False)

In [68]:
data_load = pd.read_csv(buffer_store_path, sep='\t')

In [69]:
data_load.head()

,s,a,r,s_,d
0,"[0.416, 0.13333333333333333, 0.159292035398230...",0.0,0.0,"[0.272, 0.18666666666666668, 0.088495575221238...",0.0
1,"[0.272, 0.18666666666666668, 0.088495575221238...",1.0,0.0,"[0.298, 0.17333333333333334, 0.120943952802359...",0.0
2,"[0.298, 0.17333333333333334, 0.120943952802359...",5.0,0.0,"[0.298, 0.17333333333333334, 0.120943952802359...",0.0
3,"[0.298, 0.17333333333333334, 0.120943952802359...",1.0,0.0,"[0.514, 0.16, 0.17994100294985252, 0.192307692...",0.0
4,"[0.514, 0.16, 0.17994100294985252, 0.192307692...",3.0,0.0,"[0.252, 0.17333333333333334, 0.103244837758112...",0.0


In [70]:
buffer.describe()

,a,r,d
count,404728.000000,404728.00000,404728.000000
mean,3.168254,0.00688,0.018281
std,1.688798,0.13132,0.133967
min,0.000000,-0.05000,0.000000
25%,2.000000,0.00000,0.000000
50%,4.000000,0.00000,0.000000
75%,5.000000,0.00000,0.000000
max,5.000000,1.00000,1.000000
